# August 13, 2020

## Preparing the Data

### Partitioning by row types

The data for this project is in the HURDAT2 format from NHC. This format contains rows of storm positions interspersed with header rows denoting which storm the subsequent position data corresponds to.

Because of this, the raw data table has a few problems:
- No column names are provided.
- Columns contain a mix of data types.
- Many rows are full of missing data for most columns.

As a result, our first goal will be to convert the data into a more usable format. We begin by importing the raw data as is into a Pandas DataFrame, `atl`. In doing so, we also assign column names which correspond to the information in the storm position data rows. We will later separate the header rows into a new DataFrame and assign them their own column names. 

In [1]:
import os
import pandas as pd

# These steps will apply both to Atlantic and Pacific datasets, so when
# we ultimately convert these steps into a function we'll allow the
# user to provide a filename. The function will always search in the
# raw data directory, so the filename alone will differentiate between
# the raw datasets.

# In this case, we'll set up the framework but assign the filename
# ourselves so we can test the code.
fn = 'Atlantic.csv'
# We're going to save the two new datasets that result from splitting
# the HURDAT file as separate files, so we'll use os.path.splitext to 
# separate the filename and extension for the raw data, then store the
# filename to use in appropriately naming the resulting files.
fn_no_ext = os.path.splitext(fn)[0]

# This list of column names applies to the position data rows.
header = ['date', 'time', 'recordID', 'status', 'lat', 'lon', 'maxSustWind', 'minPressure', 'extNE34', 'extSE34', 'extSW34', 'extNW34', 'extNE50', 'extSE50', 'extSW50', 'extNW50', 'extNE64', 'extSE64', 'extSW64', 'extNW64']

# Import data from raw data folder using our column names, and verify 
# that we got the right data.
hurdat = pd.read_csv(f'../data/01_raw/{fn}', names = header)
hurdat.head()

,date,time,recordID,status,lat,lon,maxSustWind,minPressure,extNE34,extSE34,extSW34,extNW34,extNE50,extSE50,extSW50,extNW50,extNE64,extSE64,extSW64,extNW64
0,AL011851,UNNAMED,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18510625,0000,,HU,28.0N,94.8W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,18510625,0600,,HU,28.0N,95.4W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,18510625,1200,,HU,28.0N,96.0W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,18510625,1800,,HU,28.1N,96.5W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


Based on the [NHC documentation](https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-nov2019.pdf) for the HURDAT2 database, column names are as follows:

- `date`: timecode for the position entry, format `YYYYMMDD`
- `time`: timecode for the position entry, format `HHMM` in 24-Hr UTC
- `recordID`: special designation for significant position entries. Can be empty or contain values:
 - **C**: closest approach to coast when not followed by landfall
 - **G**: genesis
 - **I**: intensity peak in both pressure and wind
 - **L**: landfall
 - **P**: minimum pressure
 - **R**: additional intensity detail during rapid changes
 - **S**: change of status
 - **T**: additional track/position detail
 - **W**: maximum wind speed
- `status`: tropical depression, tropical storm, hurricane, extratropical cyclone, subtropical depression, subtropical storm, low pressure system, tropical wave, or disturbance
- `lat`: latitude of center of storm
- `lon`: longitude of center of storm
- `maxSustWind`: maximum sustained wind
- `minPressure`: minimum central pressure
- `extDDXX`: extent of `XX` nautical mile per hour (knots) winds in the `DD` cardinal direction quadrant

Next, we want to separate the rows of `atl` into two new DataFrames, `storms` for header rows and `positions` for position data. We can do this easily by examining the `date` column. In rows containing position data, this column contains an entirely numeric string as described above. In header rows, this column contains alphabetic characters denoting the ocean basin the storm occurred in. We can use this property to easily flag header rows.

In [2]:
# We need to determine which rows go in which new DataFrame, so create
# a list that can be turned into a series and added as a new column.
header = []

# If there are only numeric characters in the date column, this is
# a position row. Otherwise, flag the row as being a header row.
for entry in hurdat['date']:
    if entry.isnumeric():
        header.append(False)
    else:
        header.append(True)

# Cast the list as a pandas series and add it as a column of DataFrame.
hurdat['header'] = pd.Series(header) 

# Create DataFrames of only header rows and only position data so we 
# can prepare each appropriately.
storms = hurdat[hurdat['header'] == True].copy() # All header columns of atl copied into new dataframe storms.
positions = hurdat[hurdat['header'] == False].copy() # All data columns of atl copied into new dataframe positions.

Now that we have our DataFrames, `storms` and `positions`, each one needs a bit more preparation.

### Storms dataframe
The header rows contain some important information, but require many fewer columns than position data rows, so we'll eliminate the excess columns and rename the remaining ones appropriately.

In [3]:
# Drop unnecessary columns, including the column indicating header rows.
storms.drop(['status', 'lat', 'lon', 'maxSustWind', 'minPressure', 'extNE34', 'extSE34', 'extSW34', 'extNW34', 'extNE50', 'extSE50', 'extSW50', 'extNW50', 'extNE64', 'extSE64', 'extSW64', 'extNW64', 'header'], axis = 1, inplace = True)
# Rename remaining columns.
storms.columns = ['stormID', 'name', 'numPositions']
# Reset indices to fill in gaps from position rows.
storms.reset_index(drop=True, inplace=True)
# Check the result.
storms.head()

,stormID,name,numPositions
0,AL011851,UNNAMED,14
1,AL021851,UNNAMED,1
2,AL031851,UNNAMED,1
3,AL041851,UNNAMED,49
4,AL051851,UNNAMED,16


These are the remaining columns, based on the [NHC documentation](https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-nov2019.pdf):

- `stormID`: an individual identifier for each storm in the form `NNXXYYYY` denoting the storm was the `XX`th storm of Hurricane Season `YYYY` in the `NN` basin (either `AL` for Atlantic, `EP` for Eastern Pacific, or `CP` for Central Pacific). Particularly useful when storms in different years share the same name, and for unnamed storms.
- `name`: name of storm.
- `numPositions`: the number of position entries in positions DataFrame corresponding to this storm

The new columns still need to be assigned the correct data types, and some require cleaning.

In [4]:
# We're going to want storms to easily be subsettable by years, so we'll
# create a new numeric column for it and fill the entries by parsing 
# the entries of the stormID column.
stormYears = []

# We can pull out the year easily since all the stormIDs share the same
# format. Note that this year corresponds to the storm season, but
# that it is possible under rare circumstances for storms to
# persist into the following year so the dates on position entries may
# not always show the year presented here.
for stormID in storms['stormID']:
    stormYears.append(stormID[4:9]) 

# Assign new year column integer dtype.
storms['year'] = pd.Series(stormYears).astype('int') 
# Reassign number of positions integer dtype.
storms['numPositions'] = storms['numPositions'].astype('int')
# Strip whitespace from name and stormID values.
storms['name'] = storms['name'].astype('str').str.strip() 
storms['stormID'] = storms['stormID'].astype('str').str.strip()

# Verify what was done to the DataFrame.
storms.head()

,stormID,name,numPositions,year
0,AL011851,UNNAMED,14,1851
1,AL021851,UNNAMED,1,1851
2,AL031851,UNNAMED,1,1851
3,AL041851,UNNAMED,49,1851
4,AL051851,UNNAMED,16,1851


### Positions dataframe
For `positions`, we need to reset the indices and reformat the latitude and longitude information to a more standardized format.

For latitudes we can write `XX.XN` as `XX.X` and `XX.XS` as `-XX.X`.
For longitudes we can write `XX.XE` as `XX.X` and `XX.XW` as `-XX.X`.

In [5]:
# Reset the index.
positions.reset_index(drop=True, inplace=True)

# Create lists to be used as new series for latitude and longitude. 
numLat = [] 
numLon = []
 
for cardLat in positions['lat']:
    if cardLat.find('N') != -1: 
        # For latitudes of degrees North, strip the whitespace and N.
        numLat.append(cardLat.strip(" N"))
    else: 
        # For latitudes of degrees South, strip the whitespace and S 
        # and make the value negative.
        numLat.append('-'+cardLat.strip(" S"))
    
for cardLon in positions['lon']:
    if cardLon.find('E') != -1: 
        # For longitudes of degrees East, strip the whitespace and E.
        numLon.append(cardLon.strip(" E"))
    else: 
        # For longitudes of degrees West, strip the whitespace and W 
        # and make the value negative.
        numLon.append('-'+cardLon.strip(" W"))
     
# Replace the existing longitude and latitude columns with the new ones.
positions['lat'] = pd.Series(numLat).astype('float')
positions['lon'] = pd.Series(numLon).astype('float')

Additionally, we want position entries to contain storm names and stormIDs for the sake of readability and subsettability. We can use the `numPositions` column of `storms` to populate this column easily.

In [6]:
# Create a list to be used as the names column in this DataFrame
stormNames = [] 

# For each storm in the storms DataFrame...
for i in range(len(storms)): 
    # For the number of rows indicated, add the storm name to the list.
    for j in range(storms['numPositions'][i]): 
        stormNames.append(storms['name'][i])
               
# And create the new column using the list.      
positions['name'] = pd.Series(stormNames)

# Repeat the process for storm IDs
stormIDs = []

for i in range(len(storms)):
    for j in range(storms['numPositions'][i]):
        stormIDs.append(storms['stormID'][i])
              
positions['stormID'] = pd.Series(stormIDs)

# Remove the unnecessary header indicator column.
positions.drop(columns="header", inplace = True)
# Verify what was done to the DataFrame.
positions.head()

,date,time,recordID,status,lat,lon,maxSustWind,minPressure,extNE34,extSE34,...,extNE50,extSE50,extSW50,extNW50,extNE64,extSE64,extSW64,extNW64,name,stormID
0,18510625,0000,,HU,28.0,-94.8,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,AL011851
1,18510625,0600,,HU,28.0,-95.4,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,AL011851
2,18510625,1200,,HU,28.0,-96.0,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,AL011851
3,18510625,1800,,HU,28.1,-96.5,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,AL011851
4,18510625,2100,L,HU,28.2,-96.8,80.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,UNNAMED,AL011851


## Exporting the DataFrames
We can now go ahead and export these new and cleaned DataFrames into the `/data/02_intermediate` directory.

While a number of rows in `positions` contain missing values for the columns pertaining to the wind extent columns, these data points should be retained as they contain other useful information. When working with wind extent data, for the sake of efficiency we can subset the data before working. However, the current state of the data will suffice for the cleaning step.

In [7]:
# We'll use the filename we stored after removing the extension earlier
# to create the child files in the new directory.
positions_fn = ( fn_no_ext + "_positions.csv" )
positions.to_csv(f"../data/02_intermediate/{positions_fn}", index = False)

storms_fn = ( fn_no_ext + "_storms.csv" )
storms.to_csv(f"../data/02_intermediate/{storms_fn}", index = False)

# Verify for the user which files were created.
print(f"Partitioned {fn} into:\n /data/02_intermediate/{positions_fn}\n /data/02_intermediate/{storms_fn}")

Partitioned Atlantic.csv into:
 /data/02_intermediate/Atlantic_positions.csv
 /data/02_intermediate/Atlantic_storms.csv


## Modularizing
We'll convert all of the above steps into a function in a new module, `clean_hurdat.py`. This module will exist in `/src/d02_intermediate`.

I'll then test the new function below:

In [8]:
import os, sys
import pandas as pd

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.d02_intermediate import clean_hurdat as cln

cln.partition_hurdat('Atlantic.csv')

Partitioned Atlantic.csv into:
 /data/02_intermediate/Atlantic_positions.csv
 /data/02_intermediate/Atlantic_storms.csv


Now our data is usable!